In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys 
import pdb 
import scipy
from neurosim.models.ssr import StateSpaceRealization as SSR
from neurosim.models.ssr import gen_random_model
from glob import glob
from dca.cov_util import calc_pi_from_data, calc_pi_from_cross_cov_mats, calc_cov_from_cross_cov_mats

In [3]:
sys.path.append('../..')

In [4]:
from loaders import load_sabes
from subspaces import CrossSubspaceIdentification, SubspaceIdentification, estimate_autocorrelation, BRSSID

### Scratch

In [15]:
indy_files = glob('/home/akumar/nse/neural_control/data/indy*')

In [16]:
dat = load_sabes(indy_files[0])

Processing spikes


100%|██████████| 1/1 [00:13<00:00, 13.62s/it]


In [18]:
y = np.squeeze(dat['spike_rates'])
pi = calc_pi_from_data(y, T=3)

In [28]:
ccmy = estimate_autocorrelation(y, 8)

In [46]:
calc_pi_from_cross_cov_mats(ccmy)

23.234529534801936

In [29]:
ssid = SubspaceIdentification()

In [30]:
# First verify that the PI is recovered from the canonical correlation coefficients

In [47]:
ht = ssid.form_hankel_toeplitz(ccmy, T=3)

In [49]:
cc = ht[1]

In [51]:
-0.5 * sum([np.log(1 - c**2) for c in cc])

23.234529534801784

In [27]:
# Next verify that the canonical correlation coefficients returned by psid yield the mutual informaton between neural data and behavior

In [53]:
def mutual_information(covjoint, covx, covy):
    return 0.5 * (np.linalg.slogdet(covx)[1] + np.linalg.slogdet(covy)[1] - np.linalg.slogdet(covjoint)[1])


In [54]:
z = np.squeeze(dat['behavior'])
ccmz = estimate_autocorrelation(z, 8)
ccm = estimate_autocorrelation(np.hstack([y, z]), 8)

In [67]:
mutual_information(calc_cov_from_cross_cov_mats(ccm[0:3]), calc_cov_from_cross_cov_mats(ccmy[0:3]), calc_cov_from_cross_cov_mats(ccmz[0:3]))

1.2246191502837576

In [57]:
bsid = PSIDSubspaceIdentification()

In [65]:
bht = bsid.form_hankel_toeplitz(ccm, 2, y.shape[1])

In [68]:
-0.5 * sum([np.log(1 - c**2) for c in bht[1]])

1.297904220247693

In [64]:
# Can also test on synthetic data

In [69]:
from ppmi.gaussian import gaussian_model

In [77]:
y, z, _, _, _, _, _ = gaussian_model()

In [94]:
ccmy = estimate_autocorrelation(y, 8)
ccmz = estimate_autocorrelation(z, 8)
ccm = estimate_autocorrelation(np.hstack([y, z]), 8)

In [95]:
mutual_information(calc_cov_from_cross_cov_mats(ccm[0:3]), calc_cov_from_cross_cov_mats(ccmy[0:3]), calc_cov_from_cross_cov_mats(ccmz[0:3]))

21.162377448897853

In [91]:
bht = bsid.form_hankel_toeplitz(np.transpose(ccm, (0, 2, 1)), 3, z.shape[1])

In [92]:
bht[1].shape

(40,)

In [93]:
-0.5 * sum([np.log(1 - c**2) for c in bht[1]])

12.369749093748798

### Testing on sabes lab data 

In [66]:
from loaders import load_sabes

In [67]:
#dat = load_sabes('/home/akumar/nse/neural_control/data/indy_20160624_03.mat')
dat = load_sabes('/mnt/Secondary/data/sabes/indy_20160624_03.mat')

Processing spikes


100%|██████████| 1/1 [00:04<00:00,  4.60s/it]


In [68]:
y = np.squeeze(dat['spike_rates'])
z = np.squeeze(dat['behavior'])

In [69]:
bsid2 = BRSSID()

In [70]:
z.shape

(9998, 2)

In [71]:
bsid2.identify(y, z, order=6, T=5)

(array([[ 0.99805831, -0.00414677,  0.02545099,  0.00898394,  0.00219401,
          0.00532583],
        [ 0.00214324,  1.00818618, -0.01003033,  0.03029479,  0.00111231,
          0.00136189],
        [-0.21937271,  0.08025669,  0.93497505,  0.00116754,  0.02650282,
          0.06082117],
        [-0.07810052, -0.23065443,  0.03438045,  0.90376874, -0.09465635,
          0.01327188],
        [-0.01660649, -0.04918319, -0.09774738,  0.50516505,  0.80507174,
         -0.04225985],
        [ 0.04803859,  0.00537496, -0.44641331, -0.12957076,  0.01573872,
          0.86618035]]),
 array([[ 9.15104833e-04,  1.11200811e-02, -4.92238800e-02,
         -4.15382217e-02,  3.57611097e-02, -1.68256207e-02],
        [-9.31464299e-03, -2.52089773e-03,  4.25898970e-03,
          1.64591753e-02, -1.67559620e-02,  2.92075411e-02],
        [ 4.75655057e-02, -4.72920075e-02,  2.19282339e-01,
          1.22835022e-01,  1.66911006e-01, -6.56974274e-01],
        [-5.89290681e-03,  1.76109253e-03,  4.9894631

In [26]:
# Test the KF decoding frm the bsid

In [ ]:
# Test on synthetic data

In [91]:
A, B, C = gen_random_model(size=6)

In [92]:
C = scipy.stats.ortho_group.rvs(dim=20)[:, 0:6]


In [93]:
ssr = SSR(A, B, C)

In [94]:
X, x = ssr.trajectory(T=int(1e5), return_state=True)

In [95]:
y = x[:, 0:18]
z = x[:, 18:]

In [96]:
bsid2 = BRSSID()

In [97]:
Ass,  _, _, r2 = bsid2.identify(y, z, order=6, T=5)

In [100]:
np.linalg.eigvals(Ass)

array([-0.62863086+0.j        , -0.50425209+0.3089909j ,
       -0.50425209-0.3089909j ,  0.15608834+0.51544694j,
        0.15608834-0.51544694j,  0.26256323+0.j        ])

In [99]:
np.linalg.eigvals(A)

array([-0.51242636+0.27857446j, -0.51242636-0.27857446j,
        0.13910177+0.51559358j,  0.13910177-0.51559358j,
        0.17536933+0.01531477j,  0.17536933-0.01531477j])

In [98]:
np.linalg.norm(A - Ass)

2.4086981303915054

In [54]:
np.arange(100)[5:-5 + 1]

array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
       22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89,
       90, 91, 92, 93, 94, 95])

In [53]:
xt.shape
z.shape

(100001, 2)

In [49]:
from sklearn.linear_model import LinearRegression

In [61]:
linmodel = LinearRegression().fit(z[4:-5], xt.T)

In [62]:
linmodel.score(z[4:-5], xt.T)

0.2831890412843523

In [64]:
linmodel.coef_ = C[-2:, :]

In [ ]:
linmodel.score(z, )

In [101]:
U = scipy.stats.unitary_group.rvs(10)

In [ ]:
delta = np.linspace(0, 0.5, 20)
nn = np.zeros(delta.size)

for i, alpha_ in enumerate(alpha):
    P = 